# Chapter 08-01: 모델 저장 형식

## 학습 목표
- TensorFlow/Keras 모델의 다양한 저장 형식을 이해한다
- 각 형식의 장단점을 파악하고 적절한 상황에 활용한다
- 전체 모델 저장과 가중치만 저장하는 방식을 구분하여 사용한다

## 목차
1. 저장 형식 비교
2. `.keras` 형식
3. SavedModel 형식
4. HDF5 형식
5. 가중치만 저장
6. 정리

---
### 🐣 초등학생을 위한 모델 저장 친절 설명!

#### 💾 왜 모델을 저장해야 해요?

> 💡 **비유**: 게임 저장처럼!
> 열심히 레벨 올린 캐릭터를 저장 안 하면
> 컴퓨터 끄면 사라져요! 모델도 마찬가지예요.

#### 📂 저장 형식 비교

| 형식 | 저장 내용 | 언제 써요? |
|------|----------|----------|
| **.keras** | 전체 모델 (구조+가중치+컴파일 설정) | 일반적인 경우 권장 |
| **SavedModel** | TF 그래프 포함 전체 저장 | 배포/서빙용 |
| **weights만** | 가중치만 (.weights.h5) | 구조는 코드로, 가중치만 저장 |

```python
# 저장
model.save('my_model.keras')          # 권장
model.save_weights('weights.h5')       # 가중치만

# 불러오기
model = tf.keras.models.load_model('my_model.keras')
model.load_weights('weights.h5')       # 구조는 직접 만든 후
```

> 💡 **중요**: 커스텀 레이어나 손실함수를 사용했다면
> 불러올 때 `custom_objects` 파라미터로 알려줘야 해요!


In [ ]:
# 필수 라이브러리 임포트
import tensorflow as tf
import numpy as np
import os

print(f"TensorFlow 버전: {tf.__version__}")

# 실습용 간단한 모델 생성 함수
def create_simple_model():
    """실습에 사용할 간단한 Dense 모델 생성"""
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(10,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 더미 데이터로 간단히 학습
model = create_simple_model()
X_dummy = np.random.randn(100, 10).astype(np.float32)
y_dummy = np.random.randint(0, 2, size=(100,)).astype(np.float32)
model.fit(X_dummy, y_dummy, epochs=2, verbose=0)
print("더미 모델 학습 완료")

## 1. 저장 형식 비교

| 형식 | 확장자 | 특징 | 추천 사용 시점 |
|------|--------|------|----------------|
| **Keras Native** | `.keras` | 구조 + 가중치 + 옵티마이저 상태 통합 저장, TF 2.12+ 기본 | 대부분의 Keras 프로젝트 |
| **SavedModel** | 디렉토리 | TensorFlow 공식 형식, 서빙·배포에 최적화 | TF Serving, TFLite 변환, 프로덕션 배포 |
| **HDF5** | `.h5` | 레거시 형식, Keras 2 시절 기본 | 구형 코드 호환, 외부 라이브러리 연동 |

> **권장**: 신규 프로젝트는 `.keras` 형식을 사용하고, 배포 시에는 SavedModel로 변환한다.

## 2. `.keras` 형식 저장 및 로드

`.keras` 형식은 TensorFlow 2.12부터 도입된 새로운 기본 형식으로,
모델 구조, 가중치, 컴파일 설정(옵티마이저 상태 포함)을 하나의 파일에 저장한다.

In [ ]:
# .keras 형식으로 전체 모델 저장
save_path_keras = '/tmp/my_model.keras'
model.save(save_path_keras)
print(f".keras 형식으로 저장 완료: {save_path_keras}")
print(f"파일 크기: {os.path.getsize(save_path_keras):,} bytes")

# .keras 형식에서 로드
loaded_model_keras = tf.keras.models.load_model(save_path_keras)
print("\n로드된 모델 요약:")
loaded_model_keras.summary()

# 원본 모델과 로드된 모델의 예측값이 동일한지 확인
original_pred = model.predict(X_dummy[:5], verbose=0)
loaded_pred   = loaded_model_keras.predict(X_dummy[:5], verbose=0)
print(f"\n예측값 일치 여부: {np.allclose(original_pred, loaded_pred)}")

## 3. SavedModel 형식

SavedModel은 TensorFlow의 공식 직렬화 형식으로 디렉토리 구조로 저장된다.
- `saved_model.pb`: 그래프 정의
- `variables/`: 가중치 파일
- `assets/`: 부가 파일(토크나이저 어휘집 등)

TF Serving, TFLite 변환, TensorFlow.js 변환의 기반이 된다.

In [ ]:
# SavedModel 형식으로 저장 (디렉토리)
save_path_sm = '/tmp/my_saved_model'
tf.saved_model.save(model, save_path_sm)
print(f"SavedModel 형식으로 저장 완료: {save_path_sm}")

# 저장된 디렉토리 구조 확인
for root, dirs, files in os.walk(save_path_sm):
    level = root.replace(save_path_sm, '').count(os.sep)
    indent = '  ' * level
    print(f"{indent}{os.path.basename(root)}/")
    for f in files:
        print(f"{indent}  {f}")

# SavedModel 로드
loaded_sm = tf.saved_model.load(save_path_sm)
print(f"\nSavedModel 로드 완료: {type(loaded_sm)}")

# Keras 모델로 로드하려면 load_model 사용
loaded_sm_as_keras = tf.keras.models.load_model(save_path_sm)
pred_sm = loaded_sm_as_keras.predict(X_dummy[:5], verbose=0)
print(f"SavedModel 예측값 일치: {np.allclose(original_pred, pred_sm)}")

## 4. HDF5 형식

HDF5(`.h5`)는 Keras 2 시절의 기본 형식이다.
현재는 레거시 지원 목적으로 유지되며, 신규 프로젝트에는 `.keras` 형식을 권장한다.

In [ ]:
# HDF5 형식으로 저장
save_path_h5 = '/tmp/my_model.h5'
model.save(save_path_h5)  # 확장자 .h5 → HDF5 형식 자동 선택
print(f"HDF5 형식으로 저장 완료: {save_path_h5}")
print(f"파일 크기: {os.path.getsize(save_path_h5):,} bytes")

# HDF5 로드
loaded_h5 = tf.keras.models.load_model(save_path_h5)
pred_h5 = loaded_h5.predict(X_dummy[:5], verbose=0)
print(f"HDF5 예측값 일치: {np.allclose(original_pred, pred_h5)}")

# 형식별 파일 크기 비교
print("\n=== 파일 크기 비교 ===")
print(f".keras : {os.path.getsize(save_path_keras):>10,} bytes")
print(f".h5    : {os.path.getsize(save_path_h5):>10,} bytes")

## 5. 가중치만 저장

모델 구조는 코드에 있고 가중치만 저장/복원하고 싶을 때 사용한다.
파일 크기가 작고 플렉서블한 로드가 가능하다.

In [ ]:
# 방법 1: get_weights() / set_weights() - NumPy 배열 리스트
weights_list = model.get_weights()
print(f"가중치 레이어 수: {len(weights_list)}")
for i, w in enumerate(weights_list):
    print(f"  레이어 {i}: shape={w.shape}, dtype={w.dtype}")

# 새 모델에 가중치 복원
new_model = create_simple_model()
new_model.set_weights(weights_list)
pred_set = new_model.predict(X_dummy[:5], verbose=0)
print(f"\nset_weights 예측값 일치: {np.allclose(original_pred, pred_set)}")

# 방법 2: save_weights / load_weights - 체크포인트 형식
weights_path = '/tmp/my_weights'
model.save_weights(weights_path)  # .index / .data-00000-of-00001 파일 생성
print(f"\n가중치 저장 완료: {weights_path}")

new_model2 = create_simple_model()
new_model2.load_weights(weights_path)
pred_lw = new_model2.predict(X_dummy[:5], verbose=0)
print(f"load_weights 예측값 일치: {np.allclose(original_pred, pred_lw)}")

# 방법 3: save_weights + .h5 확장자
model.save_weights('/tmp/my_weights.weights.h5')
print("HDF5 가중치 파일 저장 완료")

## 6. 정리 - 사용 시점 가이드

### 형식 선택 기준

| 상황 | 권장 형식 |
|------|-----------|
| 일반 Keras 개발 (저장/로드) | `.keras` |
| TF Serving으로 REST API 배포 | SavedModel |
| TFLite 모바일 변환 | SavedModel → TFLite 변환 |
| 학습 중 체크포인트 | `save_weights` (체크포인트) |
| 레거시 시스템 호환 | `.h5` |
| 가중치만 이식 (다른 구조에) | `get_weights` / `set_weights` |

### 핵심 API 요약

```python
# 전체 모델 저장
model.save('model.keras')                    # .keras 형식
model.save('saved_model_dir/')               # SavedModel 형식
model.save('model.h5')                       # HDF5 형식

# 전체 모델 로드
model = tf.keras.models.load_model('model.keras')

# 가중치만 저장/로드
model.save_weights('weights')                # 체크포인트
model.load_weights('weights')                # 체크포인트 복원
```